In [6]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
import rich

In [7]:
load_dotenv()

True

In [8]:
OPENROUTER_API_KEY  = os.getenv("OPENROUTER_API_KEY")
OPENROUTER_BASE_URL = os.getenv("OPENROUTER_BASE_URL")
if not OPENROUTER_API_KEY:
    raise RuntimeError("Missing OPENROUTER_API_KEY in .env")
if not OPENROUTER_BASE_URL:
    raise RuntimeError("Missing OPENROUTER_BASE_URL in .env")

In [9]:
client = OpenAI(api_key=OPENROUTER_API_KEY, base_url=OPENROUTER_BASE_URL)

In [10]:
# Listing 3.1
calculator_tool_definition = {
    "type": "function",
    "function": {
        "name": "calculator",
        "description": "Perform basic arithmetic operations between two numbers.",
        "parameters": {
            "type": "object",
            "properties": {
                "operator": {
                    "type": "string",
                    "description": "Arithmetic operation to perform",
                    "enum": ["add", "subtract", "multiply", "divide"]
                },
                "first_number": {
                    "type": "number",
                    "description": "First number for the calculation"
                },
                "second_number": {
                    "type": "number",
                    "description": "Second number for the calculation"
                }
            },
            "required": ["operator", "first_number", "second_number"],
        }
    }
}

In [11]:
# Listing 3.2
def calculator(operator: str, first_number: float, second_number: float) -> float:
   if operator == 'add':
       return first_number + second_number
   elif operator == 'subtract':
       return first_number - second_number
   elif operator == 'multiply':
       return first_number * second_number
   elif operator == 'divide':
       if second_number == 0:
           raise ValueError("Cannot divide by zero")
       return first_number / second_number
   else:
       raise ValueError(f"Unsupported operator: {operator}")

In [12]:
# Listing 3.3
tools = [calculator_tool_definition]

response_without_tool = client.chat.completions.create(
        model='openai/gpt-4o-mini',
        messages=[{"role": "user", "content": "What is the capital of South Korea?"}],
        tools=tools
)

In [13]:
rich.print(response_without_tool)

ChatCompletion(
    id='gen-1765356966-kV064GuI7uiFIPNrLAR6',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='The capital of South Korea is Seoul.',
                refusal=None,
                role='assistant',
                annotations=None,
                audio=None,
                function_call=None,
                tool_calls=None,
                reasoning=None
            ),
            native_finish_reason='stop'
        )
    ],
    created=1765356966,
    model='openai/gpt-4o-mini',
    object='chat.completion',
    service_tier=None,
    system_fingerprint='fp_a460d7e2b7',
    usage=CompletionUsage(
        completion_tokens=9,
        prompt_tokens=93,
        total_tokens=102,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=None,
            audio_tokens=None,
            reasoning_tokens=0,
            rejected_prediction_tokens=None,
            image_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0, video_tokens=0),
        cost=1.935e-05,
        is_byok=False,
        cost_details={
            'upstream_inference_cost': None,
            'upstream_inference_prompt_cost': 1.395e-05,
            'upstream_inference_completions_cost': 5.4e-06
        }
    ),
    provider='OpenAI'
)

In [14]:
# Plain print for easy copying
print(response_without_tool)

ChatCompletion(id='gen-1765356966-kV064GuI7uiFIPNrLAR6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of South Korea is Seoul.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None), native_finish_reason='stop')], created=1765356966, model='openai/gpt-4o-mini', object='chat.completion', service_tier=None, system_fingerprint='fp_a460d7e2b7', usage=CompletionUsage(completion_tokens=9, prompt_tokens=93, total_tokens=102, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=0, rejected_prediction_tokens=None, image_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0, video_tokens=0), cost=1.935e-05, is_byok=False, cost_details={'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 1.395e-05, 'upstream_inference_completions_cost': 5.4e-06}), provider='O

In [15]:
print(response_without_tool.choices[0].message.content)

The capital of South Korea is Seoul.


In [16]:
print(response_without_tool.choices[0].message.tool_calls)

None


In [17]:
response_with_tool = client.chat.completions.create(
        model='openai/gpt-4o-mini',
        messages=[{"role": "user", "content": "What is 1234 x 5678?"}],
        tools=tools
)

In [18]:
print(response_with_tool)

ChatCompletion(id='gen-1765356967-9BVay4QBhc9DHut6WMe0', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_3T4gaxGCw630pg85o1Oqbbvu', function=Function(arguments='{"operator":"multiply","first_number":1234,"second_number":5678}', name='calculator'), type='function', index=0)], reasoning=None), native_finish_reason='tool_calls')], created=1765356967, model='openai/gpt-4o-mini', object='chat.completion', service_tier=None, system_fingerprint='fp_a460d7e2b7', usage=CompletionUsage(completion_tokens=26, prompt_tokens=95, total_tokens=121, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=0, rejected_prediction_tokens=None, image_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0, video_token

In [19]:
print(response_with_tool.choices[0].message.content)

In [20]:
print(response_with_tool.choices[0].message.tool_calls)

[ChatCompletionMessageFunctionToolCall(id='call_3T4gaxGCw630pg85o1Oqbbvu', function=Function(arguments='{"operator":"multiply","first_number":1234,"second_number":5678}', name='calculator'), type='function', index=0)]


In [27]:
# Listing 3.4
ai_message = response_with_tool.choices[0].message

if ai_message.tool_calls:
   for tool_call in ai_message.tool_calls:
       function_name = tool_call.function.name
       function_args = json.loads(tool_call.function.arguments)

       print(f"Tool call detected:")
       print(f"  Function name: {function_name}")
       print(f"  Arguments: {function_args}")

       if function_name == "calculator":
           result = calculator(**function_args)
           print(f"  Result: {result}")

Tool call detected:
  Function name: calculator
  Arguments: {'operator': 'multiply', 'first_number': 1234, 'second_number': 5678}
  Result: 7006652


In [22]:
# Listing 3.5
messages = []
messages.append({"role": "user", "content": "What is 1234 x 5678?"})

response_with_tool = client.chat.completions.create(
    model='openai/gpt-4o-mini',
    messages=messages,
    tools=tools
)

ai_message = response_with_tool.choices[0].message

messages.append({
    "role": "assistant",
    "content": ai_message.content,
    "tool_calls": ai_message.tool_calls
})

In [23]:
if ai_message.tool_calls:
    for tool_call in ai_message.tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        if function_name == "calculator":
            result = calculator(**function_args)

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(result)
            })

In [24]:
#print the updated messages
rich.print(messages)

[
    {'role': 'user', 'content': 'What is 1234 x 5678?'},
    {
        'role': 'assistant',
        'content': '',
        'tool_calls': [
            ChatCompletionMessageFunctionToolCall(
                id='call_QfqAKFsG3NNpz4wc5gjG0gmh',
                function=Function(
                    arguments='{"operator":"multiply","first_number":1234,"second_number":5678}',
                    name='calculator'
                ),
                type='function',
                index=0
            )
        ]
    },
    {'role': 'tool', 'tool_call_id': 'call_QfqAKFsG3NNpz4wc5gjG0gmh', 'content': '7006652'}
]

In [25]:
final_response = client.chat.completions.create(
    model='openai/gpt-4o-mini',
    messages=messages
)

In [26]:
print("Messages:", messages)
print("Final Answer:", final_response.choices[0].message.content)


Messages: [{'role': 'user', 'content': 'What is 1234 x 5678?'}, {'role': 'assistant', 'content': '', 'tool_calls': [ChatCompletionMessageFunctionToolCall(id='call_QfqAKFsG3NNpz4wc5gjG0gmh', function=Function(arguments='{"operator":"multiply","first_number":1234,"second_number":5678}', name='calculator'), type='function', index=0)]}, {'role': 'tool', 'tool_call_id': 'call_QfqAKFsG3NNpz4wc5gjG0gmh', 'content': '7006652'}]
Final Answer: 1234 × 5678 = 7,006,652
